# Bin Packing Problem with Conflicts

### Algorithm 1: ILS para o PBPC (KLB, ItBL) 

```
S ← ConstroiSoluçãoInicial();
K ← Número de bins em S;
enquanto K ≥ KLB and S é viável, faça: 
  ReduzNúmeroBins(S);
  K ← K − 1;
  ItPERT ← 0;
  enquanto ItPERT ≤ ItMAX and S não é viável, faça: 
    S ← BuscaLocal(S);
    S ← EjectionChains(S);
    se ItBL sucessivas BL e EC sem melhora entao˜
      S ← Perturbação(S);
      ItPERT ← ItPERT + 1;
      ItBL ← 0;
      fim se
    fim enquanto
fim enquanto
```

## Algoritmo de construção

In [1]:
def first_fit_decreasing(itens, BIN_TOTAL_SPACE):
    sortedItens = sorted(itens, key=lambda k: k["value"], reverse=True)
    solution = [] #array of bins
    for item in sortedItens:
        if len(solution) == 0:
            solution.append({"itens":[item], "space_left":BIN_TOTAL_SPACE-item["value"]})
            continue
        for bins in solution:
            if bins["space_left"] > item["value"] and not hasConflict(item, bins):
                bins["itens"].append(item)
                bins["space_left"] -= item["value"]
            else:
                solution.append({"itens":[item], "space_left":BIN_TOTAL_SPACE-item["value"]})
    return solution
                
def hasConflict(item, bins):
    return False

In [ ]:
%load_ext ipython_unittest

In [ ]:
%%unittest_testcase
def test_first_fit(self):
    BIN_TOTAL_SPACE = 5
    fixture = [
                {"value":5},
                {"value":4},
                {"value":3},
                {"value":1},
                {"value":2}
              ]
    
    solutionExpected = [
                        {"itens":[{"value":5}], "space_left": 0},
                        {"itens":[{"value":4},{"value":1}], "space_left": 0},
                        {"itens":[{"value":3},{"value":2}], "space_left": 0},
                     ]
    
    S = first_fit_decreasing(fixture, BIN_TOTAL_SPACE)
    assert S == solutionExpected

In [ ]:
fixture = [
            {"value":5},
            {"value":4},
            {"value":3},
            {"value":1},
            {"value":2}
          ]

s = first_fit_decreasing(fixture, 5)
s